# Example: MMNL Assortment Optimization

This notebook demonstrates how to use the algorithm implemented in this repository for assortment optimization under the **Mixed Multinomial Logit (MMNL)** choice model.

It shows how to:
1. Configure and run assortment optimization for an unconstrained scenario.
2. Run different optimization procedures, such as a simple heuristic (Revenue-Ordered) and an exact method (Conic Optimization).
3. Incorporate constraints, specifically a cardinality constraint, into the optimization problem.
4. Interpret the results provided by the algorithms.

## 1. Import Required Modules


In [ ]:
import numpy as np  
import sys
import os


notebook_path = os.getcwd() 
project_root = os.path.dirname(notebook_path)  # get the project root directory
sys.path.append(project_root)

from generator.mmnl_data_generator import *
from generator.constraint import *
from models.mmnl_functions import *
from heuristic.mmnl_heuristic import *
from heuristic.general_heuristic import *
from heuristic.utils import *

print("✅ Modules imported successfully.")

✅ Modules imported successfully.


## 2. Unconstrained Assortment Optimization

In this section, we solve the assortment optimization problem without any additional constraints. 

### 2.1: Define Experiment Configuration and Generate Data
- There are many data generation methods in generator.mmnl_data_generator, you can choose and give the corresponding parameters
- The random seed is fixed to ensure that the results are reproducible. 
- We found that performance was poor for certain random seeds, so we chose these seeds to demonstrate the performance of the heuristic algorithm.

In [ ]:
# --- Configuration ---
choice_model = 'mmnl'
num_cus_type = 5 # number of customer types
num_prod = 50  # number of products
rev_type = 'RS2' # revenue type

worst_seeds = [3, 55, 73, 79, 88] # define the worst-performing seeds
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# --- Data Generation --
# Generate the data needed for the MMNL model
# The seed is fixed to ensure the generated data is always the same.
data = mmnl_data_v0_lognorm(num_prod, num_cus_type, rev_type, worst_seeds[0])

# The revenue function is what we want to maximize.
# There is to create two versions: one for the GPU (if available) and one for the CPU.
revenue_fn_cpu = get_revenue_function_mmnl(device, data)


We can inspect the generated data to understand the components of the MMNL model:
- u: A matrix representing the utility of each product for each customer type.
- price: The price of each product.
- v0: The utility of the "no-purchase" option for each customer type.
- omega: The probability of each customer type.

In [5]:
np.set_printoptions(suppress=True)
print("the utility :", np.round(data.u, 2))
print("the price :", np.round(data.price, 2))
print("the no-purchase option utility :", np.round(data.v0, 2))
print("the choice probability :", np.round(data.omega, 2))

the utility : [[  456.12   456.12   456.12   456.12   456.12   456.12   456.12   456.12
    456.12   456.12   456.12   456.12   456.12   456.12   456.12   456.12
    456.12   456.12   456.12   456.12   456.12   456.12   456.12   456.12
    456.12     0.       0.       0.       0.       0.       0.       0.
      0.       0.       0.       0.       0.       0.       0.       0.
      0.       0.       0.       0.       0.       0.       0.       0.
      0.       0.  ]
 [    3.51     3.51     3.51     3.51     3.51     3.51     3.51     3.51
      3.51     3.51     3.51     3.51     3.51     3.51     3.51     3.51
      3.51     3.51     3.51     3.51     3.51     3.51     3.51     3.51
      3.51     0.18     0.18     0.18     0.18     0.18     0.18     0.18
      0.18     0.18     0.18     0.18     0.18     0.18     0.18     0.18
      0.18     0.18     0.18     0.18     0.18     0.18     0.18     0.18
      0.18     0.18]
 [    0.26     0.26     0.26     0.26     0.26     0.26     0.

### 2.2: Perform Optimization
We will now use two different methods to find the optimal assortment:

1. Revenue-Ordered Heuristic: It selects all products with revenue greater than a threshold. And it allows to provide data that is not sorted by price.

2. Conic Optimization: It is an exact method that guarantees finding the globally optimal assortment using the conic integer formulation <sup>[1]</sup>  . It can use a warm-start for efficiency and support additional linear constraints.



#### Method 1: Revenue-Ordered(RO) Heuristic

In [6]:
# The function returns the best revenue, the number of products, the corresponding assortment.
rev_order, k, best_ass = revenue_order(num_prod, choice_model, data)
rev_order = revenue_fn_cpu(best_ass)[0]
print(f"The revenue obtained by RO: {rev_order}")
print(f"The optimal assortment obtained by RO: {best_ass}")

The revenue obtained by RO: 0.37606145287201576
The optimal assortment obtained by RO: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]


#### Method 1: Conic Optimization

In [7]:
# This function solves the assortment problem using a conic formulation.
# The 'cardinality=num_prod' argument here means there is no actual cardinality constraint (can choose up to all products).
rev_conic, ass, info = Conic_mmnl_warm_start(coef=data.omega, uti=data.u, v_i0=data.v0, p=data.price,cardinality=num_prod)
        
if ass is None:
    rev_conic
else:
    rev_conic = revenue_fn_cpu(ass)[0]

print(f"The revenue obtained by conic: {rev_conic}")
print(f"The optimal assortment obtained by conic: {ass}")

Set parameter LicenseID to value 2686707


The revenue obtained by conic: 0.43266108767833406
The optimal assortment obtained by conic: [1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]


- Allows the use of warm-start to improve computational efficiency. Function x0_vto_x0_vector can be used to convert vectors of different forms into standard assortment representations.


In [8]:
x0_vec = to_x0_vector(best_ass, n=num_prod, cardinality=num_prod)

rev_conic, ass, info = Conic_mmnl_warm_start(coef=data.omega, uti=data.u, v_i0=data.v0, p=data.price,cardinality=num_prod)
        
if ass is None:
    rev_conic
else:
    rev_conic = revenue_fn_cpu(ass)[0]
    
print(f"The revenue obtained by conic: {rev_conic}")
print(f"The optimal assortment obtained by conic: {ass}")

The revenue obtained by conic: 0.43266108767833406
The optimal assortment obtained by conic: [1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]


## 3. Constrained Assortment Optimization

 We introduce a cardinality constraint, which limits the maximum number of products that can be included in the assortment.

### 3.1: Define Constraint and Generate Data

In [ ]:
# --- Configuration for Cardinality Constrained Problem ---
choice_model = 'mmnl'
num_cus_type = 5
num_prod = 50
rev_type = 'RS2'

cap_rate = 0.1
cap = int(np.ceil(num_prod * cap_rate)) # Max number of products in assortment

worst_seeds = [55, 69, 73, 80, 88] # define the worst-performing seeds


# --- Define the cardinality constraint Ax <= B ---
A, B = cardinality(num_prod, cap)
data = mmnl_data_v0_lognorm(num_prod, num_cus_type, rev_type, worst_seeds[0])
revenue_fn, revenue_fn_cpu = get_revenue_function_mmnl(data)

In [13]:
print("the utility :", np.round(data.u, 2))
print("the price :", np.round(data.price, 2))
print("the no-purchase option utility :", np.round(data.v0, 2))
print("the choice probability :", np.round(data.omega, 2))
print("Constraint matrix A:", A)
print("Constraint vector B:", B)

the utility : [[ 12.51  12.51  12.51  12.51  12.51  12.51  12.51  12.51  12.51  12.51
   12.51  12.51  12.51  12.51  12.51  12.51  12.51  12.51  12.51  12.51
   12.51  12.51  12.51  12.51  12.51   0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.  ]
 [  0.4    0.4    0.4    0.4    0.4    0.4    0.4    0.4    0.4    0.4
    0.4    0.4    0.4    0.4    0.4    0.4    0.4    0.4    0.4    0.4
    0.4    0.4    0.4    0.4    0.4   77.45  77.45  77.45  77.45  77.45
   77.45  77.45  77.45  77.45  77.45  77.45  77.45  77.45  77.45  77.45
   77.45  77.45  77.45  77.45  77.45  77.45  77.45  77.45  77.45  77.45]
 [  0.02   0.02   0.02   0.02   0.02   0.02   0.02   0.02   0.02   0.02
    0.02   0.02   0.02   0.02   0.02   0.02   0.02   0.02   0.02   0.02
    0.02   0.02   0.02   0.02   0.02   1.17   1.17   1.17   1.17   1.17
    1.17   1.17   1.17   1.17   1.17   1.17   1.17   1

### 3.2: Perform Constrained Optimization
With the constraint Ax <= B defined, we now solve the problem again.

In [11]:
# We add the constraint matrices A and B to the function call.
rev_order, k, best_ass = revenue_order(num_prod, choice_model, data,A=A,B=B)
rev_order = revenue_fn_cpu(best_ass)[0]
print(f"The revenue obtained by RO: {rev_order}")
print(f"The optimal assortment obtained by RO: {best_ass}")
print(f"Whether the constraints are met: {sum(best_ass) <= cap}")

The revenue obtained by RO: 0.5585538114503161
The optimal assortment obtained by RO: [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
Whether the constraints are met: True


In [12]:
# We add the constraint matrices A and B to the function call.
rev_conic, ass, info = Conic_mmnl_warm_start(coef=data.omega, uti=data.u, v_i0=data.v0, p=data.price,cardinality=cap,A=A,b=B)
        
if ass is None:
    rev_conic
else:
    rev_conic = revenue_fn_cpu(ass)[0]

print(f"The revenue obtained by conic: {rev_conic}")
print(f"The optimal assortment obtained by conic: {ass}")
print(f"Whether the constraints are met: {sum(ass) <= cap}")

The revenue obtained by conic: 0.6295539852367963
The optimal assortment obtained by conic: [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
Whether the constraints are met: True


# References

[1] Şen A, Atamtürk A, Kaminsky P. A conic integer optimization approach to the constrained assortment problem under the mixed multinomial logit model[J]. Operations Research, 2018, 66(4): 994-1003.